# H20 automl: Porto Seguro (Classification) # 2

Based on: https://github.com/h2oai/h2o-tutorials/blob/master/h2o-world-2017/automl/Python/automl_binary_classification_product_backorders.ipynb

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "AUCPR"  # specify your evaluation metric
stopping = "AUCPR"  # specify your stopping metric
target = "target" # name of target or label variable (just happens to be called "target" for this dataset)

## Import

In [2]:
# Import packages

from h2o.automl import H2OAutoML

import h2o
import numpy as np
import pandas as pd

In [3]:
# Import data
train_df = pd.read_csv("porto_train.csv")
test_df = pd.read_csv("porto_test.csv")

In [4]:
train_df.shape

(476170, 60)

In [5]:
test_df.shape

(119042, 59)

In [6]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_372"; OpenJDK Runtime Environment (build 1.8.0_372-b07); OpenJDK 64-Bit Server VM (build 25.372-b07, mixed mode)
  Starting server from /home/stever7/.local/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpz_l2m9iq
  JVM stdout: /tmp/tmpz_l2m9iq/h2o_stever7_started_from_python.out
  JVM stderr: /tmp/tmpz_l2m9iq/h2o_stever7_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,"1 year, 5 months and 18 days !!!"
H2O_cluster_name:,H2O_from_python_stever7_h9xk8y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.63 Gb
H2O_cluster_total_cores:,64
H2O_cluster_allowed_cores:,64
H2O_cluster_status:,"locked, healthy"


## Tidy

In [7]:
train_df = train_df.replace(-1, np.nan)
test_df = test_df.replace(-1, np.nan)

## Transform

In [8]:
# Convert categorical variables to "category" data type (after converting to h20 dataframe - easier)
cat_vars = [col for col in train_df.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [9]:
train_df = h2o.H2OFrame(train_df)
test_df = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
for col in cat_vars:
    test_df[col] = test_df[col].asfactor()
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_df[col] = train_df[col].asfactor()

In [11]:
predictors = train_df.columns
predictors.remove(target)  # remove "target"
predictors.remove("id")  # remove "id" (should not be a predictor)
predictors.remove("fold")  # remove "fold" (not be a predictor)

## Visualize

(skipped)

## Model

In [12]:
%%time

aml = H2OAutoML(
    max_runtime_secs=time_limit, 
    seed=1, 
    sort_metric=metric,
    stopping_metric=stopping
)

CPU times: user 3.29 ms, sys: 1.5 ms, total: 4.79 ms
Wall time: 31.2 ms


In [13]:
%%time

aml.train(x=predictors, y=target, training_frame=train_df)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

04:29:28.618: GBM_lr_annealing_selection_AutoML_1_20230804_32940 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

CPU times: user 24.3 s, sys: 2.07 s, total: 26.4 s
Wall time: 1h 4s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_3_AutoML_1_20230804_32940

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.035220481155405194
RMSE: 0.18767120491808326
LogLoss: 0.1465732076381303
Null degrees of freedom: 9852
Residual degrees of freedom: 9813
Null deviance: 3200.8068545728493
Residual deviance: 2888.371629716996
AIC: 2968.371629716996
AUC: 0.7778363432174876
AUCPR: 0.16445171067383865
Gini: 0.5556726864349752

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06210558944096188: 


,,0,1,Error,Rate
0,0,8722.0,754.0,0.0796,(754.0/9476.0)
1,1,229.0,148.0,0.6074,(229.0/377.0)
2,Total,8951.0,902.0,0.0998,(983.0/9853.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.062106,0.231431,147.0
1,max f2,0.056625,0.324796,163.0
2,max f0point5,0.114200,0.261438,48.0
3,max accuracy,0.227058,0.962042,4.0
4,max precision,0.426447,1.000000,0.0
5,max recall,0.014750,1.000000,382.0
6,max specificity,0.426447,1.000000,0.0
7,max absolute_mcc,0.057921,0.213063,159.0
8,max min_per_class_accuracy,0.039843,0.705044,236.0
9,max mean_per_class_accuracy,0.041091,0.708325,230.0



Gains/Lift Table: Avg response rate:  3.83 %, avg score:  3.68 %


...

,,0,1,Error,Rate
0,0,407197.0,51614.0,0.1125,(51614.0/458811.0)
1,1,13035.0,4324.0,0.7509,(13035.0/17359.0)
2,Total,420232.0,55938.0,0.1358,(64649.0/476170.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.057180,0.117986,193.0
1,max f2,0.039531,0.199875,259.0
2,max f0point5,0.084152,0.099241,132.0
3,max accuracy,0.425466,0.963553,8.0
4,max precision,0.592581,1.000000,0.0
5,max recall,0.008811,1.000000,399.0
6,max specificity,0.592581,1.000000,0.0
7,max absolute_mcc,0.044536,0.081681,238.0
8,max min_per_class_accuracy,0.035197,0.598826,280.0
9,max mean_per_class_accuracy,0.034760,0.600495,282.0



Gains/Lift Table: Avg response rate:  3.65 %, avg score:  3.65 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010001,0.112004,3.525327,3.525327,0.128517,0.141985,0.128517,0.141985,0.035255,0.035255,252.532660,252.532660,0.026210
1,2,0.020001,0.094759,2.586392,3.055859,0.094288,0.102287,0.111403,0.122136,0.025866,0.061121,158.639158,205.585909,0.042676
2,3,0.030002,0.085318,2.361738,2.824486,0.086098,0.089677,0.102968,0.111316,0.023619,0.084740,136.173841,182.448553,0.056809
3,4,0.040000,0.078905,2.114488,2.647014,0.077085,0.081939,0.096498,0.103973,0.021142,0.105882,111.448793,164.701409,0.068374
4,5,0.050001,0.074072,2.102523,2.538111,0.076648,0.076368,0.092528,0.098452,0.021027,0.126908,110.252322,153.811134,0.079817
5,6,0.100000,0.060039,1.851528,2.194827,0.067498,0.066150,0.080013,0.082301,0.092574,0.219483,85.152802,119.482689,0.124003
6,7,0.150001,0.052471,1.549596,1.979747,0.056491,0.055969,0.072173,0.073524,0.077481,0.296964,54.959589,97.974688,0.152523
7,8,0.200000,0.047332,1.375684,1.828734,0.050151,0.049766,0.066667,0.067584,0.068783,0.365747,37.568417,82.873437,0.172018
8,9,0.300000,0.040332,1.222997,1.626822,0.044585,0.043579,0.059307,0.059582,0.122300,0.488047,22.299672,62.682182,0.195161
9,10,0.400000,0.035450,1.054208,1.483668,0.038432,0.037785,0.054088,0.054133,0.105421,0.593467,5.420819,48.366841,0.200787


In [14]:
%%time

lb = aml.leaderboard

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 15.5 µs


In [15]:
lb.head(rows=lb.nrows)

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_3_AutoML_1_20230804_32940,0.0672582,0.641096,0.151841,0.431701,0.186381,0.034738
StackedEnsemble_AllModels_4_AutoML_1_20230804_32940,0.0672579,0.641106,0.151839,0.431296,0.186381,0.0347379
StackedEnsemble_BestOfFamily_4_AutoML_1_20230804_32940,0.0664389,0.638656,0.151986,0.434445,0.186419,0.034752
XGBoost_grid_1_AutoML_1_20230804_32940_model_16,0.0657561,0.63725,0.152101,0.437428,0.186452,0.0347642
XGBoost_grid_1_AutoML_1_20230804_32940_model_23,0.0654968,0.636616,0.152144,0.43116,0.186465,0.0347692
StackedEnsemble_AllModels_2_AutoML_1_20230804_32940,0.0654858,0.637552,0.152091,0.434554,0.186456,0.0347658
StackedEnsemble_BestOfFamily_3_AutoML_1_20230804_32940,0.0653121,0.636994,0.152126,0.430847,0.186464,0.034769
GBM_grid_1_AutoML_1_20230804_32940_model_7,0.0649473,0.634313,0.152286,0.431045,0.186488,0.0347777
GBM_grid_1_AutoML_1_20230804_32940_model_20,0.0648433,0.632125,0.152534,0.435165,0.186529,0.0347929
XGBoost_grid_1_AutoML_1_20230804_32940_model_20,0.0646487,0.63199,0.152519,0.440402,0.186542,0.0347981


In [16]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_3_AutoML_1_20230804_32940

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.035220481155405194
RMSE: 0.18767120491808326
LogLoss: 0.1465732076381303
Null degrees of freedom: 9852
Residual degrees of freedom: 9813
Null deviance: 3200.8068545728493
Residual deviance: 2888.371629716996
AIC: 2968.371629716996
AUC: 0.7778363432174876
AUCPR: 0.16445171067383865
Gini: 0.5556726864349752

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06210558944096188: 


,,0,1,Error,Rate
0,0,8722.0,754.0,0.0796,(754.0/9476.0)
1,1,229.0,148.0,0.6074,(229.0/377.0)
2,Total,8951.0,902.0,0.0998,(983.0/9853.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.062106,0.231431,147.0
1,max f2,0.056625,0.324796,163.0
2,max f0point5,0.114200,0.261438,48.0
3,max accuracy,0.227058,0.962042,4.0
4,max precision,0.426447,1.000000,0.0
5,max recall,0.014750,1.000000,382.0
6,max specificity,0.426447,1.000000,0.0
7,max absolute_mcc,0.057921,0.213063,159.0
8,max min_per_class_accuracy,0.039843,0.705044,236.0
9,max mean_per_class_accuracy,0.041091,0.708325,230.0



Gains/Lift Table: Avg response rate:  3.83 %, avg score:  3.68 %


...

,,0,1,Error,Rate
0,0,407197.0,51614.0,0.1125,(51614.0/458811.0)
1,1,13035.0,4324.0,0.7509,(13035.0/17359.0)
2,Total,420232.0,55938.0,0.1358,(64649.0/476170.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.057180,0.117986,193.0
1,max f2,0.039531,0.199875,259.0
2,max f0point5,0.084152,0.099241,132.0
3,max accuracy,0.425466,0.963553,8.0
4,max precision,0.592581,1.000000,0.0
5,max recall,0.008811,1.000000,399.0
6,max specificity,0.592581,1.000000,0.0
7,max absolute_mcc,0.044536,0.081681,238.0
8,max min_per_class_accuracy,0.035197,0.598826,280.0
9,max mean_per_class_accuracy,0.034760,0.600495,282.0



Gains/Lift Table: Avg response rate:  3.65 %, avg score:  3.65 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010001,0.112004,3.525327,3.525327,0.128517,0.141985,0.128517,0.141985,0.035255,0.035255,252.532660,252.532660,0.026210
1,2,0.020001,0.094759,2.586392,3.055859,0.094288,0.102287,0.111403,0.122136,0.025866,0.061121,158.639158,205.585909,0.042676
2,3,0.030002,0.085318,2.361738,2.824486,0.086098,0.089677,0.102968,0.111316,0.023619,0.084740,136.173841,182.448553,0.056809
3,4,0.040000,0.078905,2.114488,2.647014,0.077085,0.081939,0.096498,0.103973,0.021142,0.105882,111.448793,164.701409,0.068374
4,5,0.050001,0.074072,2.102523,2.538111,0.076648,0.076368,0.092528,0.098452,0.021027,0.126908,110.252322,153.811134,0.079817
5,6,0.100000,0.060039,1.851528,2.194827,0.067498,0.066150,0.080013,0.082301,0.092574,0.219483,85.152802,119.482689,0.124003
6,7,0.150001,0.052471,1.549596,1.979747,0.056491,0.055969,0.072173,0.073524,0.077481,0.296964,54.959589,97.974688,0.152523
7,8,0.200000,0.047332,1.375684,1.828734,0.050151,0.049766,0.066667,0.067584,0.068783,0.365747,37.568417,82.873437,0.172018
8,9,0.300000,0.040332,1.222997,1.626822,0.044585,0.043579,0.059307,0.059582,0.122300,0.488047,22.299672,62.682182,0.195161
9,10,0.400000,0.035450,1.054208,1.483668,0.038432,0.037785,0.054088,0.054133,0.105421,0.593467,5.420819,48.366841,0.200787


In [17]:
perf = aml.leader.model_performance(test_df)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.034717887965771375
RMSE: 0.18632736773155836
LogLoss: 0.15197236004535086
Null degrees of freedom: 119041
Residual degrees of freedom: 119002
Null deviance: 37231.77510716302
Residual deviance: 36182.18736903732
AIC: 36262.18736903732
AUC: 0.6350923458573843
AUCPR: 0.0667778498478644
Gini: 0.27018469171476855

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.05690794335361876: 


,,0,1,Error,Rate
0,0,101241.0,13466.0,0.1174,(13466.0/114707.0)
1,1,3222.0,1113.0,0.7433,(3222.0/4335.0)
2,Total,104463.0,14579.0,0.1402,(16688.0/119042.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.056908,0.117691,184.0
1,max f2,0.039742,0.197328,249.0
2,max f0point5,0.093893,0.103549,100.0
3,max accuracy,0.417833,0.963618,1.0
4,max precision,0.463168,1.000000,0.0
5,max recall,0.010597,1.000000,397.0
6,max specificity,0.463168,1.000000,0.0
7,max absolute_mcc,0.051561,0.082527,201.0
8,max min_per_class_accuracy,0.035389,0.591926,271.0
9,max mean_per_class_accuracy,0.038210,0.596829,256.0



Gains/Lift Table: Avg response rate:  3.64 %, avg score:  3.69 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010005,0.112975,3.481579,3.481579,0.126784,0.140678,0.126784,0.140678,0.034833,0.034833,248.157936,248.157936,0.025766
1,2,0.020001,0.095501,2.976829,3.229310,0.108403,0.103396,0.117598,0.122045,0.029758,0.064591,197.682882,222.931009,0.046274
2,3,0.030006,0.086214,2.144284,2.867533,0.078086,0.090488,0.104423,0.111523,0.021453,0.086044,114.428398,186.753346,0.058155
3,4,0.040003,0.080120,1.915324,2.629581,0.069748,0.082924,0.095758,0.104376,0.019146,0.105190,91.532397,162.958107,0.067651
4,5,0.050008,0.075028,2.029000,2.509424,0.073887,0.077425,0.091382,0.098984,0.020300,0.125490,102.899989,150.942448,0.078335
5,6,0.100007,0.060674,1.868544,2.189011,0.068044,0.066917,0.079714,0.082952,0.093426,0.218916,86.854350,118.901091,0.123403
6,7,0.150006,0.052940,1.619404,1.999153,0.058972,0.056507,0.072801,0.074138,0.080969,0.299885,61.940437,99.915269,0.155543
7,8,0.200005,0.047765,1.296446,1.823483,0.047211,0.050212,0.066403,0.068156,0.064821,0.364706,29.644623,82.348346,0.170925
8,9,0.300003,0.040783,1.174184,1.607056,0.042759,0.044031,0.058522,0.060115,0.117416,0.482122,17.418351,60.705620,0.189002
9,10,0.400002,0.035886,0.973488,1.448668,0.035450,0.038215,0.052754,0.054640,0.097347,0.579469,-2.651190,44.866750,0.186250


## Communicate

(skipped)